Credits to Тимур Петров, ФКН ВШЭ

## TDD

### Паттерн разработки TDD

TDD - test-driven developement или разработка через тестирование. Достигается соблюдением трех правил.

**Три правила TDD**:

 - Продакшн-код можно писать только для починки падающего теста
 - В тесте нужно писать ровно столько кода, сколько необходимо чтобы он упал. Ошибки компиляции считаются падениями теста.
 - В прод можно написать ровно столько кода, сколько требуется для починки дного падающего теста.


 Получается следйющий пайплайн - пишем падающий тест, пишем код чтобы тест не падал, рефакторим код так, чтобы тесты не падали. Повторяем до сходимости.

1. [Test Driven Development: By Example 1st Edition](https://www.eecs.yorku.ca/course_archive/2003-04/W/3311/sectionM/case_studies/money/KentBeck_TDD_byexample.pdf)
2. [On Growing Object Oriented Software, Guided by Tests](https://www.amazon.com/Growing-Object-Oriented-Software-Guided-Tests/dp/0321503627)


### Каты

![](https://karate.by/uploads/posts/2010-10/kkk0001s.png)

Каты - упражнения по программированию, помогающие отточить навыки путем многократного повторнеия. Концепция взята из японских боевых искусств. Подробнее про них можно почитать в книжке [The Pragmatic Programmer](https://pragprog.com/titles/tpp20/the-pragmatic-programmer-20th-anniversary-edition/)



**Ката Greeter**

Данную кату надо выполнять строго по пунктам, не заглядывая вперед:

- Создайте класс `Greeter`, у которого есть метод `greet` принимающий на вход имя и возвращающий "Hello <имя>".
- Метод `greet` должен убирать лишние пробелы - в начале и в конце имени
- Метод `greet` должен возвращать ошибку если имя - пустая строка (или строка с пробелами)
- Метод `greet` возвращает "Good evening <имя>" если текущее время - 18:00-22:59

## Первый тест

Для автоматизированного тестирования написано много фреймворков на разных языках. Короткий список для python:

* [unittest](https://docs.python.org/3/library/unittest.html)
* [nose2](https://docs.nose2.io/en/latest/)
* [pytest](https://docs.pytest.org/en/latest/)

В рамках лекции мы остановимся на `pytest`. Почему он? Банально - он удобнее (если детально углубляться, глобально ничем не отличаются, просто в pytest уже достаточно много всего сделано из коробки)

In [ ]:
!pip install ipytest pytest coverage

In [ ]:
import pytest
import ipytest
import coverage
ipytest.autoconfig()
__file__ = "testing_and_logging_lection.ipynb"

#### Как pytest находит тесты:

1. Рекурсивно находит все python-файлы в текущей директории
2. Оставляет только файлы вида `test_*.py` и `*_test.py`
3. В этих файлах
  1. Находит все функции с префиксом `test`
  2. Находит все методв с префиксом `test` внутри классов с префиксом `Test`. У классов не должно быть метода `__init__`
  
Поведение можно модифицировать. [Подробнее в документации](https://docs.pytest.org/en/stable/goodpractices.html#test-discovery)

Напишем минимальный тест (который, естественно, упадет)

In [ ]:
%%ipytest -q
def test_greeter():
    Greeter()

`pytest` выводит отчет, в котором можно посмотреть сколько у нас всего тестов, какие из них упали и по какой причине.

Теперь сделаем так чтобы тест проходил

TDD выглядит следующим образом:

1. Написали тест

2. Написали кусок кода

3. Проверили, что проходит тест

4. PROFIT

In [ ]:
class Greeter:
    pass

In [ ]:
%%ipytest -q
def test_greeter():
    assert Greeter()

Сначала мы пишем тесты на реализацию что должно проверять, а только потом делаем реализацию.

Теперь наконец-то напишем нормальный тест, воспользовавшись основной фишкой `Pytest` - `assert`. `Pytest` находит все вызовы `assert` в коде тестов, а затем переписывает этот код так, чтобы в случае падения пользователь мог получить удобный дифф и трейсбек.

- [Демки разных ассертов](https://docs.pytest.org/en/stable/example/reportingdemo.html#tbreportdemo)
- [Цикл статей про то, как это работает](https://www.pythoninsight.com/2018/01/assertion-rewriting-in-pytest-part-1/)

Базово assert работает достаточно просто: мы вызываем некоторое выражение, которое должно выдавать True/False. Получилось True - мы молодцы, False - нет, роняем

In [ ]:
%%ipytest -q
def test_greeter():
    name = "Mike"
    assert Greeter().greet(name) == name

Еще одна итерация TDD:

In [ ]:
class Greeter:
    def greet(self, name):
        return name

In [ ]:
%%ipytest -q
def test_greeter():
    name = "Mike"
    assert Greeter().greet(name) == name

##  Параметризация

Наша реализация представляет собой немного не то что мы хотели. В чем проблема? А в том, что у нас написан всего 1 тест, который можно обработать изощренно:

In [ ]:
class Greeter:
    def greet(self, name):
        return "Mike"

In [ ]:
%%ipytest -q
def test_greeter():
    name = "Mike"
    assert Greeter().greet(name) == name

Наверное стоит добавить больше разных тестов, чтобы вводы были разные.

Чтобы не копировать один и тот же тест (и не плодить 100500 тестов), можно воспользоваться параметризацией:

Посмотрим что это такое на примере:

In [ ]:
%%ipytest -q
import pytest


@pytest.mark.parametrize("a, b, c", [
    (1, 2, 3),
    (3, 4, 7),
    (5, 6, 11),
])
def test_sample_parametrization(a, b, c):
    assert a + b == c

Давайте рассмотрим что из себя оно представляет:

Конкректно у нас фигурирует тут: `pytest.mark.parametrize`
Декоратор, который позволяет параметризировать (использовать несколько наборов данных) на тестовую функцию.

Каждая комбинация значений будет автоматически передаваться в тестовую функцию и pytest выполнит для каждого набора тестовых данных.



```python
@pytest.mark.parametrize("<аргумент1>, <аргумент2>, <аргумент3>, ...", [<набор_Значений_для_аргумента_1>, <набор_значений_для_аргумента_2>, <набор_значений_для_аргумента_3>, ...]
ИЛИ [
    (<набор_Значений_для_аргумента_1>, <набор_значений_для_аргумента_2>, <набор_значений_для_аргумента_3>, ...),
    (<набор_Значений_для_аргумента_1>, <набор_значений_для_аргумента_2>, <набор_значений_для_аргумента_3>, ...),
    (<набор_Значений_для_аргумента_1>, <набор_значений_для_аргумента_2>, <набор_значений_для_аргумента_3>, ...)
])
def test_<название теста>(<аргумент1>, <аргумент2>, <аргумент3>, ...):

    <подготовка или демонстрация тестового сценария>

    assert <проверяемое выражение>

```


* аргументы в фикстурах -- переменные и для каждой переменной должно быть значение типа object (то есть любой объект)

Если не будет хоть одного значения тест завершится ошибкой с проверкой на количество аргументов и значений  `assert len(param.values) != len(argnames)`

In [ ]:
%%ipytest -q

@pytest.mark.parametrize("a, b, c", [1, 2])
def test_sample(a, b, c):
    assert a + b == 3

Видим отчеты пайтеста во всей красе. Починим тесты:

In [ ]:
%%ipytest -q
test_cases = [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")]

@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting

In [ ]:
class Greeter:
    def greet(self, name):
        return "Hello " + name

In [ ]:
%%ipytest -q
test_cases = [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")]

@pytest.mark.parametrize("name, greeting", test_cases)
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting

Ура, мы выполнили первый пункт!

Перейдем к следующему пункту нашего задания:

- Метод `greet` должен убирать лишние пробелы - в начале и в конце имени

Опять же, напишем тест:

In [ ]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeting = greeter.greet(" Mike")
    assert not greeting.startswith(" ")

Обратим внимание что тест проходит и возникает соблазн продолжить работу. Однако если посмотреть на тест внимательно - можно увидеть в нем ошибку. (Какая?)


Чтобы не наступать на такие грабли существует **правило** - только что написанный тест должен падать, при чем именно из-за того поведения, которое этот тест должен было покрыть.


Вы можете писать тесты на уже существующий код - в таком случае они могут не падать т.к. код уже работает как надо. Тогда есть два варианта:
* Сделать в продовом коде баг чтобы тест упал
* Обратить проверяемое условие в тесте

Поправим наш тест:

In [ ]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeted_name = greeter.greet(" Mike").split(" ", 1)[1]
    assert not greeted_name.startswith(" ")

In [ ]:
class Greeter:
    def greet(self, name):
        if name.startswith(" "):
            name = name[1:]
        return "Hello " + name

In [ ]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeted_name = greeter.greet(" Mike").split(" ", 1)[1]

    assert not greeted_name.startswith(" ")

Перечитаем наше задание:
* Метод greet должен убирать лишние пробелы - в начале и **в конце имени**

Видимо нам нужно расширить тест:

In [ ]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeted_name = greeter.greet(" Mike ").split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Починим тест

In [ ]:
class Greeter:
    def greet(self, name):
        if name.startswith(" "):
            name = name[1:]
        if name.endswith(" "):
            name = name[:1]
        return "Hello " + name

In [ ]:
%%ipytest -q

def test_spaces():
    greeter = Greeter()
    greeted_name = greeter.greet(" Mike ").split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Наш тест все еще недостаточно хорош. Хороший набор тестов должен покрывать разные граничные условия и заходить во все ветки исполнения кода. Параметризуем наш тест так чтобы покрывал как можно ветвей исполнения кода:

In [ ]:
%%ipytest -q

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "])
def test_spaces(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Можно давать имена отдельным наборам параметров - тогда будет удобнее читать вывод пайтеста

In [ ]:
%%ipytest -q

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

In [ ]:
class Greeter:
    def greet(self, name):
        while name.startswith(" "):
            name = name[1:]
        while name.endswith(" "):
            name = name[:1]
        return "Hello " + name

In [ ]:
%%ipytest -q

@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Код кажется многословным! (как будто можем проще, правда?) После рефакторинга необходимо его подчистить!

In [ ]:
class Greeter:
    def greet(self, name):
        return "Hello " + name.strip()

In [ ]:
%%ipytest -q


@pytest.mark.parametrize("name, greeting", [("Mike", "Hello Mike"), ("John", "Hello John"), ("Greg", "Hello Greg")])
def test_greeter(name, greeting):
    assert Greeter().greet(name) == greeting


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_spaces(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Шикарно, выполнили 2 пункта из 4, а также познакомились с параметризацией, можем сразу много кода написать

## Моки и фикстуры

Mock (с англ - подделка, заглушка и тд) -- объекты, которые создаются для замены реальных объектов в процессе тестирования.

Основная идея мока заключается в том, чтобы изолировать тестируемую часть системы от других компонентов, которые могут быть сложными, медленными или недоступными на момент тестирования.

Большая потребность их возникает тогда, когда мы пишем большое и сложное приложение в котором имеется очень сложная логика и невозможно протестировать as appropriate, уже сразу написанный код.

То есть логика следующая: мы не знаем, что у нас будет за объект/мы не хотим к нему обращаться. Чтобы протестировать функционал, нам необходимо создать какой-то фейк, который ***симулирует*** поведение нужного объекта. Вот это и есть Mock

Fixture (c англ - крепление, зацепка) -- уже подготовленные наборы данных, которые имеют состояние. В данном случае у нас уже есть какие-то данные, которые мы не хотим постоянно собирать (потому что собирать на каждый тест - долго/дорого). Идея: взять и зафиксировать их

Fixture != Mock

Рассмотрим типичную ситуацию когда прибегают к мокам:

**Проблема**
Есть класс который взаимодействует со внешней системой, количество запросов ограниченно. Нужно уметь тестировать логику в которой фигурирует вызов этой внешней системы независя от реальных вызовов.

In [ ]:
class UserService:
    def __init__(self, db):
        self._db = db

    def get_user_status(self, user_id):
        user_data = self._db.get_user(user_id)  # представим что реализация и вызов этого метода нет
        if user_data.get('active'):
            return f"User {user_id} is active"
        return f"User {user_id} is deactivated"

Для того чтобы протестировать этот функционал приходят на помощь мокирование этого вызова. Мы пропишем в тесте что мы ожидаем от этого вызова и вернем значение:

In [ ]:
!pip install mock

In [ ]:
%%ipytest -q

from mock import Mock

@pytest.mark.parametrize("user_id, db_response, expected_message", [
    (1, {"active": True}, f"User 1 is active"),   # Активный пользователь
    (2, {"active": False}, f"User 2 is deactivated"),  # Неактивный пользователь
    (3, {}, "User 3 is deactivated"),  # Пустые данные пользователя
])

def test_user_service(user_id, db_response, expected_message):
    mock_db = Mock()

    mock_db.get_user.return_value = db_response

    service = UserService(mock_db)

    result = service.get_user_status(user_id)
    assert result == expected_message

    mock_db.get_user.assert_called_once_with(user_id)

Давайте рассмотрим что мы описали тут:

* `mock_db = Mock()` -- создали пустой объект мока

* `mock_db.get_user.return_value` -- Перегрузка мока. В буквальном смысле: при вызове метода `get` объекта `db` верни ответ `db_response`.

Структура записи такова:

`<экземпляр класса мока>.<подменяющий вызывающий метод>.return_value = <значение>`


А что если мы хотим проверять вызов ошибки?

Давайте добавим в класс обработку ошибок:

In [ ]:
class UserService:
    def __init__(self, db):
        self._db = db

    def get_user_status(self, user_id):
        try:
            user_data = self._db.get_user(user_id)
            if user_data.get('active'):
                return f"User {user_id} is active"
            return f"User {user_id} is deactivated"
        except Exception as e:
            return f"Error retrieving user {user_id}: {str(e)}"

Напишем сценарий в котором при вызове метода будет вызываться ошибка:

In [ ]:
%%ipytest -q

@pytest.mark.parametrize("user_id, db_response, expected_message, should_raise", [
    (4, None, "Error retrieving user 4: Database connection failed", True),
])
def test_user_service(user_id, db_response, expected_message, should_raise):
    mock_db = Mock()

    if should_raise:
        mock_db.get_user.side_effect = Exception("Database connection failed")
    else:
        mock_db.get_user.return_value = db_response

    service = UserService(mock_db)
    result = service.get_user_status(user_id)
    assert result == expected_message
    assert mock_db.get_user.call_count == 1

Все что мы заготавливали мы оставляем все как и есть. Но только заменяем `return_value` на `side_effect` и прописываем какую ошибку мы ждали:

Структура записи такова:

`<экземпляр класса мока>.<подменяющий вызывающий метод>.side_effect = <значение>`

Наблюдаемая проблема:

* У нас появился Boiler Print в тестах от которого нужно избавляться!

Фикстуры так же могут чистить использованный инвентарь за создаваемым объектом в конце теста и иметь разный scope - например создваться на каждый тест, модуль или тред, запускающий тесты. [Подробнее в документации](https://docs.pytest.org/en/stable/fixture.html).

Scopes (скоупы) у фикстур бывают:
* function (по умолчанию)
* class -- внутри класса
* module -- создаваемый внутри модуля

Давайте перепишем это в фикстуру!

In [ ]:
%%ipytest -q


@pytest.fixture(scope='function')
def mock_db(request):
    mock_db = Mock()

    db_response, should_raise = request.param

    if should_raise:
        mock_db.get_user.side_effect = Exception("Database connection failed")
    else:
        mock_db.get_user.return_value = db_response

    return mock_db


In [ ]:
%%ipytest -q


@pytest.mark.parametrize("mock_db, user_id, expected_message", [
    (({"active": True}, False), 1, "User 1 is active"),
    (({"active": False}, False), 2, "User 2 is deactivated"),
    (({}, False), 3, "User 3 is deactivated"),
    ((None, True), 4, "Error retrieving user 4: Database connection failed"),
], indirect=["mock_db"])
def test_user_service(mock_db, user_id, expected_message):
    service = UserService(mock_db)

    result = service.get_user_status(user_id)

    assert result == expected_message

    assert mock_db.get_user.call_count == 1

Что мы тут сделали?
* создали фикстуру в которой происходит подготовка всего необходимого для возвращения значений
* немного поправили входные параметры в нашей параметризации: то есть мы прокинули туда Mock объект и прописали логику при которой будет вызываться новое поведение в `request.params`

Такой механизм называется "непрямая параметризация" - `indirect parametrization`

В `pytest` объект request автоматически передается в фикстуры, когда они вызываются.
Например, с его помощью можно получить параметры, переданные в фикстуру, или информацию о тестовой функции, которая используетcя оной.
Когда используется параметризация тестов с флагом `indirect`, параметры передаются через фикстуру. Внутри фикстуры они становятся доступными через `request.param`

Для БД фикстура может выглядеть примерно так:

In [ ]:
%%ipytest -q

class DBConnection:
    pass

class TestDB:
    def init_db(self):
        print("init db")

    def get_connection(self):
        return DBConnection()

    def shutdown(self):
        print("close db")

@pytest.fixture(scope="module")
def db_connection():
    db = TestDB()
    db.init_db()
    try:
        yield db.get_connection()
    finally:
        db.shutdown()

def test_db_1(db_connection):
    assert db_connection

def test_db_2(db_connection):
    assert db_connection

Заключение:

* Мок может меняться и быть в тесте, а в свою очередь фикстура имеет свою структуру и не может меняться
* Моки легко параметризировать и писать различные тест кейсы на каждый случай
* Легко "вшивать фикстуры в тесты и прокидывать тестовые аргументы в параметры фикстуры тем самым задавая им свое поведение

#### Вернемся к решению каты:

Тесты нужно рефакторить. В основном главная задача тестов:
* проверять код, покрывая все возможные случаи на берегу (чтобы не отправить багу в production и при этом не потерять деньги)
* понятность и простота тестов (нельзя долго писать тесты на определенную фичу, это стоит очень дорого и ресурсы не предрасположены чтобы заниматься ими. Помните: тесты хоть и не помогают в зарабатыванию денег, но при этом без них есть шанс потерять много денег)
* documentation as a code (когда вы придете работать куда-нибудь, они вам помогут быстрее разобраться в коде, чем вы сами сидели и читали код)

**[Первая проблема]**:
* Имена тестов не очень информативны. Если упадет тест `test_greater` - будет не совсем понятно что именно тестировалось и что надо чинить. В целом имена тестам надо давать как можно более подробные - тесты вызываются автоматически, автоматике длинна имени безразлична, а вот человеку, читающему выхлоп пайтеста, лучше предоставить как можно больше информации.

[Статья на тему](https://enterprisecraftsmanship.com/posts/you-naming-tests-wrong)

In [ ]:
%%ipytest -q

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(name):
    greeter = Greeter()
    greeted_name = greeter.greet(name).split(" ", 1)[1]

    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

**[Вторая проблема]**

В обоих тестах мы создаем `greeter`. Это привожит к дублированию кода. Кроме того, на практике вместо `greeter` у нас может быть какой-нибудь тяжелый объект типа базы даных или контекста сессии, который надо каждый раз инициализировать и чистить. Решить эти проблемы нам поможет механизм фикстур:

In [ ]:
%%ipytest -q

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

Так же в `pytest` есть разные встроенные фикстуры. [Список лежит здесь](https://docs.pytest.org/en/stable/fixture.html). Наиболее интересные:
* monkeypatch - временно можифицирует методы классов, модулей и т.д.
* testdir - создает верменную директорию для каждого теста, которую потом чистит

## Тестирование исключений, патчинг и работа с тестами где фигурируют флоты

Возвращаемся к катам. Третья часть:

- Метод `greet` должен возвращать ошибку если имя - пустая строка (или строка с пробелами)

Проблема: мы должны тестировать исключение, но при этом мы не должны его ловить стандартным try-catch в тесте а просто создать среду в котором будет **ожидаться** падение теста. Для этого необходимо это создать в тесте. На помощь приходит ```pytest.raises``` (который ожидает, что БУДЕТ ошибка)

In [ ]:
%%ipytest -q

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

def test_greet_raises_value_error_on_empty_string(greeter):
    with pytest.raises(Exception): # (*)
        greeter.greet("")

# (*) обычно вы тут ждать определенную ошибку которую вы сами создали, к примеру вы написали класс ошибки которым отнаследовались от базового класса Exception

По тексту отчета видим, что тест ожидал исключения, но его не было. Починим тест:

In [ ]:
class Greeter:
    def greet(self, name):
        name = name.strip()
        if not name:
            raise ValueError("Empty name!")
        return "Hello " + name

И дополнительно параметризуем:

In [ ]:
%%ipytest -q

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

@pytest.mark.parametrize("name", ["", "   ", "  ", " "])
def test_greet_raises_value_error_on_empty_string(greeter, name):
    with pytest.raises(ValueError):
        greeter.greet(name)

Ура, осталась последняя ката:

- Метод `greet` возвращает "Good evening <имя>" если текущее время - 18:00-22:59

 ## Работа с тестами при участии времени

Пишем тест:

In [ ]:
%%ipytest -q

import datetime

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

def test_greeting_is_good_evening_in_evening(monkeypatch, greeter):
    fake_time =  datetime.datetime(2020, 11, 10, 19)
    class mydatetime:
        @classmethod
        def now(cls):
            return fake_time

    monkeypatch.setattr(datetime, 'datetime', mydatetime)
    assert greeter.greet("Mike").startswith("Good evening")

Изменим код, и разберем тест выше:

In [ ]:
import datetime

class Greeter:
    def greet(self, name):
        name = name.strip()
        if not name:
            raise ValueError("Empty name!")
        hour = datetime.datetime.now().hour
        if 18 <= hour <= 22:
            return "Good evening " + name
        return "Hello " + name

Посмотрим, что со старыми тестами (они не будут падать, но как будто что-то не то)

In [ ]:
%%ipytest -q

@pytest.fixture(scope="module")
def greeter():
    yield Greeter()

@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")

@pytest.mark.parametrize("name", ["", "   ", "  ", " "])
def test_greet_raises_value_error_on_empty_string(greeter, name):
    with pytest.raises(ValueError):
        greeter.greet(name)

Будет падать, потому что время текущее не матчится. Зададим дефолтное время.

In [ ]:
%%ipytest -q
import datetime


@pytest.fixture(scope="function")
def set_time(monkeypatch):
    def set_time_(time):
        class mydatetime:
            @classmethod
            def now(cls):
                return time

        monkeypatch.setattr(datetime, 'datetime', mydatetime)
    yield set_time_


@pytest.fixture(scope="function")
def set_day_time(set_time):
    yield set_time(datetime.datetime(2020, 10, 10, 10))


@pytest.mark.parametrize("name", ["Mike", "John", "Greg"])
def test_greet_returns_name_with_greeting(set_day_time, greeter, name):
    assert Greeter().greet(name) == "Hello " + name


@pytest.mark.parametrize("name", ["Mike", " Mike", "Mike ", " Mike ", "  Mike", "  Mike  "],
                         ids=["no spaces", "left space", "right space",
                              "two-side space", "double space", "two-sided double space"])
def test_greet_removes_leading_and_trailing_spaces_from_name(set_day_time, greeter, name):
    greeted_name = greeter.greet(name).split(" ", 1)[1]
    assert not greeted_name.startswith(" ") and not greeted_name.endswith(" ")


def test_greeting_is_good_evening_in_evening(set_time, monkeypatch, greeter):
    set_time(datetime.datetime(2020, 11, 10, 19))
    assert greeter.greet("Mike").startswith("Good evening")

Ну вроде все, у нас все удалось, поздравляю!

## Работа с плавающей точкой

Сравнение `float` сталось за кадром - разберем его отдельно.
 Из-за ошибок округления `float` трудно сравнивать через `==`

In [ ]:
%%ipytest -q
def test_float():
    assert 0.1 + 0.2 == 0.3

На помощь приходит API pytest'а заиспользуем `pytest.approx`

In [ ]:
%%ipytest -q
def test_float():
    assert [0.1 + 0.2, 0.5] == pytest.approx([0.3, 0.5])

## Попугай дня

![](https://do-slez.com/uploads/posts/2020-02/1582813051_pesquets-dracula-parrots-birds-new-guinea-1-5e55392f17e1e__700.jpg)

Сегодня у нас очень красивый орлиный попугай (орлиный за счет его клюва) и сходит в семейство щетиноголовых попугаев (видите какая щетина у него прям)

![](https://img.theepochtimes.com/assets/uploads/2020/05/14/Dracula-Parrot-i.jpg)

Исторически живет в Новой Гвинее и его очень редко можно встретить в зоопарках из-за очень прихотливого питания (им обязательно нужны тропические фрукты для ферментации) и требований к содержанию (температура, влажность)

Еще один, к сожалению, вымирающий вид, потому что на них охотятся индейцы за их красные перья (хоть и достаточно распространен в авикультуре, но выращивать его очень сложно).